In [6]:
# imports
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI


In [7]:
# Connect to Ollama (instead of OpenAI cloud)
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')


In [9]:
# Quick test call
message = "Hello, Llama! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(
    model="llama3.2",
    messages=[{"role":"user", "content":message}]
)
print(response.choices[0].message.content)


Hello! It's great to meet you for the first time! I'm excited to chat with you and help with any questions or topics you'd like to discuss. How's your day going so far? Is there something specific on your mind that you'd like to talk about, or do you just want to get to know me a bit better?


In [11]:
# Website scraper
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    def __init__(self, url):
        """Create this Website object from the given url using BeautifulSoup"""
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"

        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()

        self.text = soup.body.get_text(separator="\n", strip=True)


In [12]:
# Test scraper
ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text[:500])  # just first 500 chars


Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to


In [13]:
# System + user prompts
system_prompt = (
    "You are an assistant that analyzes the contents of a website "
    "and provides a short summary, ignoring text that might be navigation related. "
    "Respond in markdown."
)

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}\n"
    user_prompt += (
        "The contents of this website is as follows; "
        "please provide a short summary of this website in markdown. "
        "If it includes news or announcements, then summarize these too.\n\n"
    )
    user_prompt += website.text
    return user_prompt

print(user_prompt_for(ed)[:500])  # preview


You are looking at a website titled Home - Edward Donner
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badl


In [14]:
# Messages preview
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

response = openai.chat.completions.create(model="llama3.2", messages=messages)
print(response.choices[0].message.content)


*sigh* Really? You can't even do basic math on your own? Fine. The answer to 2 + 2 is... *dramatic pause* ...4. Happy now? Can I help you with something actually challenging or just waste more of my time with simple questions?


In [15]:
# Build messages for summarization
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

messages_for(ed)


[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a

In [16]:
# Summarizer
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model="llama3.2",
        messages=messages_for(website)
    )
    return response.choices[0].message.content

summarize("https://edwarddonner.com")


'**Summary of Website**\n=================================================================\n\n### About the Owner\n\n*Ed Donner is a co-founder, CTO, and founder of AI startup Nebula.io, which applies AI to help people discover their potential.*\n*He has experience as CEO of untapt, an ACquired company, and has written about LLMs, DJing, amateur electronic music production, and hacker news in his hobbies.*\n\n### News and Announcements\n\n* Upcoming Course: "Connecting my courses – become an LLM expert and leader" (May 28, 2025)*\n* Past Events:\n\t+ June 2021 Acquisition of AI startup untapt*\n\t+ Various AIF Executive Briefings in April 2025*\n\t+ Release of The Complete Agentic AI Engineering Course* January 23, 2025*\n\t+ Hosts an LWM Workshops - Hand on Agents - resources May Not Available*\n\n### Projects\n\n*Nebula.io: Works with groundbreaking, proprietary LLMs verticalized for talent*\n\nNote that \'Connect \'with me\' link will only point to a newsletter signup instead of any

In [17]:
# Display nicely in notebook
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

display_summary("https://edwarddonner.com")


**Website Summary**
===============

### Overview

The website belongs to Edward Donner, a co-founder and CTO of Nebula.io. The platform focuses on applying AI to help people discover their potential.

### Project/Service Details

* **Nebula.io**: Applying AI for talent management using groundbreaking LLMs.
* **Untapt**: Acquired in 2021, an AI startup that was previously founded by Edward Donner.

### Upcoming Events and Courses

* **Courses**:
	+ Connecting my courses – become an LLM expert and leader (May 28, 2025)
	+ The Complete Agentic AI Engineering Course (January 23, 2025)
* **Workshops**:
	+ LLM Workshop – Hands-on with Agents (resources) (January 23, 2025)
* **News/Miscellaneous**
	+ April 21, 2025: 2025 AI Executive Briefing
	+ May 18, 2025: Connecting my courses – an online event

### Contact Information

* Email: [ed@edwarddonner.com](mailto:ed@edwarddonner.com)
* Website: www.edwarddonner.com

In [18]:
# Try more sites
display_summary("https://cnn.com")
display_summary("https://anthropic.com")


**Summary**

**Category:** Breaking News, Latest News and Videos | CNN

**Description**
The website features the latest news updates on global events, politics, business, health, entertainment, technology, sports, climate, science, and more. The website utilizes a dynamic menu system that organizes content by category, allowing users to easily navigate through various sections.

**Recent Announcements/News**

- **Majestic Moment**: A rare moment of emotion from the Chinese leader Xi during a summit with Putin.
- **Russian Pool via Reuters:** Plane carrying EU's top leader targeted by alleged Russian GPS jamming.
- **Deadly protests in Indonesia**, the conflict continues amidst resentment.
- **India's Lion population turning into deadly problems** due to growing numbers affecting communities in Gujarat.

# Anthropic Website Summary

## Overview

Anthropic is a company that builds AI models and solutions to serve humanity's long-term well-being. Its core focus is on building tools with human benefit at their foundation, aiming to show what responsible AI development looks like in practice.

### Key Products and Services

* **Claude**: An AI model that enables users to create applications and custom experiences.
	+ **Claude Opus 4.1**: The most intelligent AI model, pushing the frontier for coding and AI agents.
	+ **Sonnet 4** and **Haiku 3.5**: Other powerful models.
* **Anthropic Academy**: A learning platform that helps users build with Claude.

## News and Announcements

### Recent News

* **Claude Opus 4.1 Available**, our most intelligent AI model, is now available.
* **Talk to Claude** allows users to interact with the new model.
* **Read Announcement** provides more information about Claude Opus 4.1.

### Latest Insights and Publications

* A research lead discusses the importance of designing powerful technologies that consider societal impacts.
* Anthropic's views on AI safety and responsible scaling policy are featured.

## Call to Action

Anthropic is seeking talented individuals to join its team, focusing on building a future for safe AI. **See open roles** for more information.

## Footer Information

* Product information: Claude overview, Pricing, and Plans.
* API Platform: API overview, Developer docs, and integration with Amazon Bedrock and Google Cloud's Vertex AI.
* Cookie Policy and Terms of Service available upon request.